# 네이버 주식 코드 공유

In [18]:
## 시리얼 넘버 가져오기

import requests
from bs4 import BeautifulSoup

# header 사용.
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}
url ='https://finance.naver.com/sise/sise_group_detail.nhn?type=theme&no=310'    
    # 리츠 메인페이지
    
response = requests.get(url, headers = headers)
soup = BeautifulSoup(response.text, 'html.parser')

companyinfo = []

tr_list = soup.select("table.type_5 > tbody")

for tr in tr_list:
    a_list = tr.select('a')
    for i in range(len(a_list)):
        if i % 3 == 0:
            name = a_list[i].text             # 기업명
            code = a_list[i].attrs["href"].strip('/item/main.nhn?code=')   # 시리얼번호
            companyinfo.append([name, code])

companyinfo    # [name, code]

[['코람코에너지리츠', '357120'],
 ['ESR켄달스퀘어리츠', '365550'],
 ['롯데리츠', '330590'],
 ['이지스레지던스리츠', '350520'],
 ['NH프라임리츠', '338100'],
 ['이리츠코크렙', '088260'],
 ['제이알글로벌리츠', '348950'],
 ['이지스밸류리츠', '334890'],
 ['미래에셋맵스리츠', '357250'],
 ['모두투어리츠', '204210'],
 ['신한알파리츠', '293940'],
 ['케이탑리츠', '145270'],
 ['에이리츠', '140910']]

In [19]:
## 마지막 페이지 구하기 (약 1~2분 소요)

import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

for i in range(len(companyinfo)):
    
    if companyinfo[i][1] != '365550':       # 'ESR켄달스퀘어리츠'는 페이지가 1개 뿐이라 lastpage_btn을 지정할 수 없음
        
        chromedriver = 'C:/Users/Yoon/Pysson/chromedriver'
        driver = webdriver.Chrome(chromedriver)
        driver.get('https://finance.naver.com/item/frgn.nhn?code={}&page=1'
                   .format(companyinfo[i][1]))  # code 자동입력

        # '맨뒤' 버튼 지정
        lastpage_btn = driver.find_element_by_css_selector(
            "#content > div.section.inner_sub > table.Nnavi > tbody > tr > td.pgRR > a")

        lastpage_btn.click()        # '맨뒤' 버튼 클릭
        driver.implicitly_wait(3)

        new_url = driver.current_url   # 클릭해서 들어간 페이지의 '현재 페이지 url' 출력

        lastpage = str(new_url)[-3:].strip('e').strip('=')   # 마지막 페이지: 맨 뒤의 3개의 글자를 긁어와 "="와 'e'만 삭제함.

        companyinfo[i].append(lastpage)
        
    elif companyinfo[i][1] == '365550':
        companyinfo[i].append(1)     # 'ESR켄달스퀘어리츠'는 마지막 페이지가 1 (2021.01.08 기준)
        
        
companyinfo    # [name, code, lastpage]

[['코람코에너지리츠', '357120', '5'],
 ['ESR켄달스퀘어리츠', '365550', 1],
 ['롯데리츠', '330590', '15'],
 ['이지스레지던스리츠', '350520', '6'],
 ['NH프라임리츠', '338100', '14'],
 ['이리츠코크렙', '088260', '32'],
 ['제이알글로벌리츠', '348950', '6'],
 ['이지스밸류리츠', '334890', '6'],
 ['미래에셋맵스리츠', '357250', '6'],
 ['모두투어리츠', '204210', '53'],
 ['신한알파리츠', '293940', '30'],
 ['케이탑리츠', '145270', '110'],
 ['에이리츠', '140910', '117']]

In [20]:
## 주식별 존재하는 자료의 가장 오래된 날짜 크롤링

for i in companyinfo:
    URL = 'https://finance.naver.com/item/frgn.nhn?code={}&page={}'.format(i[1], i[2])
        # code, lastpage 자동입력
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}
    response = requests.get(URL+str(j), headers = headers)
    soup = BeautifulSoup(response.text,"html.parser")
    
    tr_list = soup.select('table[class="type2"] tr')
    temp_list = []
    
    for tr in tr_list:
        if tr.select('td.num'):
            td_list=tr.select('span.tah')
            if td_list:
                temp_list.append(td_list[0].text)
                
    i.append(temp_list[-1])

companyinfo      # [name, code, lastpage, firstdate]

[['코람코에너지리츠', '357120', '5', '2020.08.31'],
 ['ESR켄달스퀘어리츠', '365550', 1, '2020.12.23'],
 ['롯데리츠', '330590', '15', '2019.10.30'],
 ['이지스레지던스리츠', '350520', '6', '2020.08.05'],
 ['NH프라임리츠', '338100', '14', '2019.12.05'],
 ['이리츠코크렙', '088260', '32', '2018.06.27'],
 ['제이알글로벌리츠', '348950', '6', '2020.08.07'],
 ['이지스밸류리츠', '334890', '6', '2020.07.16'],
 ['미래에셋맵스리츠', '357250', '6', '2020.08.05'],
 ['모두투어리츠', '204210', '53', '2016.09.22'],
 ['신한알파리츠', '293940', '30', '2018.08.08'],
 ['케이탑리츠', '145270', '110', '2012.01.31'],
 ['에이리츠', '140910', '117', '2011.07.14']]

In [21]:
## firstdate가 2020년 이전인 경우만 추출

companyinfo2 = []

for i in companyinfo:
    if int(i[3][:4]) < 2020:
        companyinfo2.append(i)
        
companyinfo2      # 2020년 자료가 FULL로 있는 데이터만 분석 대상으로서 선정 (7개의 주식 데이터)

[['롯데리츠', '330590', '15', '2019.10.30'],
 ['NH프라임리츠', '338100', '14', '2019.12.05'],
 ['이리츠코크렙', '088260', '32', '2018.06.27'],
 ['모두투어리츠', '204210', '53', '2016.09.22'],
 ['신한알파리츠', '293940', '30', '2018.08.08'],
 ['케이탑리츠', '145270', '110', '2012.01.31'],
 ['에이리츠', '140910', '117', '2011.07.14']]

In [23]:
## 데이터 크롤링 & CSV 파일로 만들기 (약 3~4분 소요)

import csv

import requests
from bs4 import BeautifulSoup

for i in companyinfo2:

    file = open("naver_fins_{}({}).csv".format(i[1], i[0]),mode="w", encoding="utf-8",newline="")
        # code, name 자동입력
    writer = csv.writer(file)
    writer.writerow( ('date','price','amt' ) )

    j = 1

    while True:
        URL = 'https://finance.naver.com/item/frgn.nhn?code={}&page='.format(i[1])   # code 자동입력
        headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}
        response = requests.get(URL+str(j), headers = headers)
        soup = BeautifulSoup(response.text,"html.parser")
        tr_list = soup.select('table[class="type2"] tr')
        for tr in tr_list:
            if tr.select('td.num'):
                td_list=tr.select('span.tah')
                if td_list:
                    if td_list[0].text[:4] == str(2020):   # date 앞 네 글자(연도)가 2020이면 출력
                        date = td_list[0].text
                        price = td_list[1].text
                        amt = td_list[4].text
                        print(date)
                        print(price)
                        print(amt)
                        writer.writerow ( (date,price,amt ) )
        j += 1

        if j == int(i[2]):      # lastpage
            file.close()
            break

2020.12.30
5,350
553,446
2020.12.29
5,280
602,244
2020.12.28
5,400
670,608
2020.12.24
5,430
330,934
2020.12.23
5,440
398,280
2020.12.22
5,420
375,881
2020.12.21
5,460
358,002
2020.12.18
5,520
568,118
2020.12.17
5,420
500,285
2020.12.16
5,370
125,047
2020.12.15
5,370
222,980
2020.12.14
5,360
345,380
2020.12.11
5,360
481,901
2020.12.10
5,230
441,518
2020.12.09
5,230
219,074
2020.12.08
5,250
162,821
2020.12.07
5,260
395,332
2020.12.04
5,250
773,604
2020.12.03
5,270
658,336
2020.12.02
5,340
367,640
2020.12.01
5,370
252,648
2020.11.30
5,350
414,314
2020.11.27
5,400
361,309
2020.11.26
5,500
265,816
2020.11.25
5,570
292,360
2020.11.24
5,530
443,613
2020.11.23
5,510
393,579
2020.11.20
5,490
215,983
2020.11.19
5,500
500,808
2020.11.18
5,430
188,821
2020.11.17
5,390
268,012
2020.11.16
5,350
545,605
2020.11.13
5,410
416,926
2020.11.12
5,470
119,190
2020.11.11
5,470
154,161
2020.11.10
5,470
380,134
2020.11.09
5,390
214,598
2020.11.06
5,400
184,646
2020.11.05
5,440
271,291
2020.11.04
5,410
119,369


2020.08.10
4,405
91,226
2020.08.07
4,425
43,500
2020.08.06
4,430
54,240
2020.08.05
4,450
52,837
2020.08.04
4,450
39,714
2020.08.03
4,445
37,996
2020.07.31
4,455
33,172
2020.07.30
4,465
21,917
2020.07.29
4,470
53,683
2020.07.28
4,470
36,267
2020.07.27
4,470
35,640
2020.07.24
4,465
16,825
2020.07.23
4,445
61,688
2020.07.22
4,460
40,830
2020.07.21
4,430
41,825
2020.07.20
4,425
39,833
2020.07.17
4,400
82,220
2020.07.16
4,365
113,236
2020.07.15
4,425
55,379
2020.07.14
4,440
29,683
2020.07.13
4,435
33,204
2020.07.10
4,445
75,320
2020.07.09
4,440
66,048
2020.07.08
4,475
22,661
2020.07.07
4,490
59,923
2020.07.06
4,515
59,311
2020.07.03
4,510
40,319
2020.07.02
4,525
70,651
2020.07.01
4,595
34,677
2020.06.30
4,630
30,323
2020.06.29
4,600
43,873
2020.06.26
4,570
23,163
2020.06.25
4,550
25,308
2020.06.24
4,555
70,849
2020.06.23
4,560
29,188
2020.06.22
4,590
37,439
2020.06.19
4,615
65,950
2020.06.18
4,600
95,134
2020.06.17
4,650
114,063
2020.06.16
4,700
97,774
2020.06.15
4,695
57,526
2020.06.12
4,7

2020.03.05
5,900
156,783
2020.03.04
5,680
84,604
2020.03.03
5,480
75,038
2020.03.02
5,470
81,943
2020.02.28
5,500
159,613
2020.02.27
5,660
42,167
2020.02.26
5,690
49,044
2020.02.25
5,720
74,622
2020.02.24
5,720
121,947
2020.02.21
5,800
142,142
2020.02.20
5,910
37,669
2020.02.19
5,940
56,433
2020.02.18
5,890
58,225
2020.02.17
5,910
172,393
2020.02.14
6,020
229,590
2020.02.13
6,200
104,456
2020.02.12
6,250
120,619
2020.02.11
6,330
47,474
2020.02.10
6,310
30,087
2020.02.07
6,300
20,866
2020.02.06
6,300
49,567
2020.02.05
6,300
34,365
2020.02.04
6,290
32,586
2020.02.03
6,340
93,638
2020.01.31
6,140
66,893
2020.01.30
6,190
38,551
2020.01.29
6,210
23,853
2020.01.28
6,140
51,901
2020.01.23
6,260
35,856
2020.01.22
6,260
38,192
2020.01.21
6,320
48,890
2020.01.20
6,300
73,238
2020.01.17
6,250
104,066
2020.01.16
6,340
87,716
2020.01.15
6,390
63,897
2020.01.14
6,330
125,906
2020.01.13
6,410
112,977
2020.01.10
6,530
36,901
2020.01.09
6,610
29,526
2020.01.08
6,540
120,701
2020.01.07
6,560
65,422
2020

2020.10.12
6,780
44,941
2020.10.08
6,800
82,797
2020.10.07
6,790
113,912
2020.10.06
6,770
50,103
2020.10.05
6,800
81,371
2020.09.29
6,870
107,136
2020.09.28
6,730
129,260
2020.09.25
6,780
152,532
2020.09.24
6,770
113,643
2020.09.23
6,760
84,217
2020.09.22
6,760
66,342
2020.09.21
6,790
67,794
2020.09.18
6,810
70,749
2020.09.17
6,820
42,256
2020.09.16
6,840
69,471
2020.09.15
6,850
101,791
2020.09.14
6,850
107,647
2020.09.11
6,810
70,688
2020.09.10
6,780
89,087
2020.09.09
6,720
106,635
2020.09.08
6,730
166,128
2020.09.07
6,690
78,169
2020.09.04
6,650
93,620
2020.09.03
6,650
64,056
2020.09.02
6,610
42,954
2020.09.01
6,610
57,571
2020.08.31
6,610
100,002
2020.08.28
6,620
54,417
2020.08.27
6,640
70,225
2020.08.26
6,660
51,048
2020.08.25
6,660
102,853
2020.08.24
6,570
72,400
2020.08.21
6,530
79,904
2020.08.20
6,480
99,363
2020.08.19
6,550
92,742
2020.08.18
6,490
203,380
2020.08.14
6,610
94,418
2020.08.13
6,620
120,973
2020.08.12
6,630
191,119
2020.08.11
6,490
162,377
2020.08.10
6,500
108,439


2020.05.18
762
143,375
2020.05.15
758
198,413
2020.05.14
761
155,341
2020.05.13
770
121,259
2020.05.12
774
152,514
2020.05.11
789
160,701
2020.05.08
791
156,958
2020.05.07
789
130,106
2020.05.06
790
199,368
2020.05.04
792
140,636
2020.04.29
790
382,167
2020.04.28
788
352,181
2020.04.27
801
175,450
2020.04.24
800
131,996
2020.04.23
808
123,977
2020.04.22
804
213,440
2020.04.21
806
474,943
2020.04.20
810
180,040
2020.04.17
808
311,185
2020.04.16
792
476,823
2020.04.14
810
361,567
2020.04.13
805
249,147
2020.04.10
820
350,391
2020.04.09
822
222,909
2020.04.08
790
191,324
2020.04.07
783
227,614
2020.04.06
770
405,930
2020.04.03
773
133,235
2020.04.02
772
185,129
2020.04.01
760
270,500
2020.03.31
761
147,236
2020.03.30
753
138,683
2020.03.27
750
215,406
2020.03.26
750
227,407
2020.03.25
727
167,001
2020.03.24
701
153,015
2020.03.23
690
308,607
2020.03.20
709
242,776
2020.03.19
685
1,018,028
2020.03.18
713
219,017
2020.03.17
725
318,842
2020.03.16
744
125,944
2020.03.13
740
978,165
2020.03.1

In [24]:
## 주가 분석 (CSV -> dataframe)

import pandas as pd

csv_list = []

for i in companyinfo2:
    csv_list.append("naver_fins_{}({}).csv".format(i[1], i[0]))
    
for i, val in enumerate(csv_list):
    print(i, val, sep=' ')       # enumerate 출력값: csv 파일명

    
dfs = []

for i in range(len(csv_list)):
    temp = pd.read_csv(csv_list[i], header=0)
    
    ## 1000 단위 콤마 제거 - 그래야 int로 변화시킬 수 있음
    temp["price"] = temp["price"].apply(lambda x: str(x.replace(',','')))
    temp["amt"] = temp["amt"].apply(lambda x: str(x.replace(',','')))
    
    ## object -> int
    temp = temp.astype({'price': 'int', 'amt': 'int'})
    
    ## 가격변동량, 가격변동률(%) 칼럼 추가
    temp['price_diff'] = (temp.price - temp.price.shift(-1)).fillna(0)
    temp['price_change'] = ((temp.price - temp.price.shift(-1)) /temp.price.shift(-1) *100).fillna(0)
    temp = temp.astype({'price_diff': 'int'})     # price_diff의 소수점 없앰
    
    ## 거래량에 대해서도 diff, change 설정
    temp['amt_diff'] = (temp.amt - temp.amt.shift(-1)).fillna(0)
    temp['amt_change'] = ((temp.amt - temp.amt.shift(-1)) /temp.amt.shift(-1) *100).fillna(0)
    temp = temp.astype({'amt_diff': 'int'})     # diff의 소수점 없앰
    
    ## 소수점 표기 옵션 변경(소수점 2째 자리까지)
    pd.options.display.float_format = '{:.2f}'.format     
    
    dfs.append(temp)

dfs[0]    # 개별 회사로 보고싶으면 0~6 사이의 숫자 입력: enumerate 출력값 참고

0 naver_fins_330590(롯데리츠).csv
1 naver_fins_338100(NH프라임리츠).csv
2 naver_fins_088260(이리츠코크렙).csv
3 naver_fins_204210(모두투어리츠).csv
4 naver_fins_293940(신한알파리츠).csv
5 naver_fins_145270(케이탑리츠).csv
6 naver_fins_140910(에이리츠).csv


,date,price,amt,price_diff,price_change,amt_diff,amt_change
0,2020.12.30,5350,553446,70,1.33,-48798,-8.10
1,2020.12.29,5280,602244,-120,-2.22,-68364,-10.19
2,2020.12.28,5400,670608,-30,-0.55,339674,102.64
3,2020.12.24,5430,330934,-10,-0.18,-67346,-16.91
4,2020.12.23,5440,398280,20,0.37,22399,5.96
...,...,...,...,...,...,...,...
243,2020.01.08,5910,1134397,-70,-1.17,304792,36.74
244,2020.01.07,5980,829605,-80,-1.32,-175598,-17.47
245,2020.01.06,6060,1005203,-40,-0.66,457225,83.44
246,2020.01.03,6100,547978,-30,-0.49,-109396,-16.64


In [25]:
# 결측값 확인
for i in dfs:
    print(i.isnull().sum())
    
    # 결측값은 없음을 확인할 수 있음. 
    # 2020.01.02의 price_diff, price_change, amt_diff, amt_change는 이전 값(2020.01.01가 없어 0으로 표기됨)
    # 그러나 결측값으로는 처리되지는 않았음

date            0
price           0
amt             0
price_diff      0
price_change    0
amt_diff        0
amt_change      0
dtype: int64
date            0
price           0
amt             0
price_diff      0
price_change    0
amt_diff        0
amt_change      0
dtype: int64
date            0
price           0
amt             0
price_diff      0
price_change    0
amt_diff        0
amt_change      0
dtype: int64
date            0
price           0
amt             0
price_diff      0
price_change    0
amt_diff        0
amt_change      0
dtype: int64
date            0
price           0
amt             0
price_diff      0
price_change    0
amt_diff        0
amt_change      0
dtype: int64
date            0
price           0
amt             0
price_diff      0
price_change    0
amt_diff        0
amt_change      0
dtype: int64
date            0
price           0
amt             0
price_diff      0
price_change    0
amt_diff        0
amt_change      0
dtype: int64


In [26]:
## 통합 주가 분석

new_df = dfs[0] + dfs[1] + dfs[2] + dfs[3] + dfs[4] + dfs[5] + dfs[6]


## 날짜 재지정 - 재지정하지 않으면 '2020.12.302020.12.292020.12.28...' 으로 표기됨
new_df['date'] = dfs[0]['date']

## 가격변동량, 가격변화율(%)은 개별의 합과는 차이가 발생할 수 있으므로 재지정
new_df['price_diff'] = (new_df.price - new_df.price.shift(-1)).fillna(0)
new_df['price_change'] = ((new_df.price - new_df.price.shift(-1)) /new_df.price.shift(-1) *100).fillna(0)
new_df = new_df.astype({'price_diff': 'int'})     # diff의 소수점 없앰

## 거래량에 대해서도 diff, change 설정
new_df['amt_diff'] = (new_df.amt - new_df.amt.shift(-1)).fillna(0)
new_df['amt_change'] = ((new_df.amt - new_df.amt.shift(-1)) /new_df.amt.shift(-1) *100).fillna(0)
new_df = new_df.astype({'amt_diff': 'int'})     # diff의 소수점 없앰

## 소수점 표기 옵션 변경(소수점 2째 자리까지)
pd.options.display.float_format = '{:.2f}'.format

# 칼럼 순서 재설정
new_df = new_df[['date', 'price', 'price_diff', 'price_change', 'amt', 'amt_diff', 'amt_change']]

new_df  # 7개의 주식의 통합 DB

,date,price,price_diff,price_change,amt,amt_diff,amt_change
0,2020.12.30,33115,-85,-0.26,1228001,-52572,-4.11
1,2020.12.29,33200,-1800,-5.14,1280573,-457593,-26.33
2,2020.12.28,35000,-195,-0.55,1738166,-894424,-33.98
3,2020.12.24,35195,260,0.74,2632590,1397797,113.20
4,2020.12.23,34935,-265,-0.75,1234793,-215943,-14.89
...,...,...,...,...,...,...,...
243,2020.01.08,36006,-423,-1.16,2413801,1011615,72.15
244,2020.01.07,36429,-176,-0.48,1402186,-771533,-35.49
245,2020.01.06,36605,-550,-1.48,2173719,651560,42.80
246,2020.01.03,37155,-176,-0.47,1522159,68075,4.68


In [27]:
# MIN, MAX, MEAN 분석 - price_change, amt_change

print(
    f'가격변동율(%) MIN: {format(new_df.price_change.min(), ".2f")}',  # 소수점 2번째 자리까지 출력
    f'가격변동율(%) MAX: {format(new_df.price_change.max(), ".2f")}',
    f'가격변동율(%) MEAN: {format(new_df.price_change.mean(), ".2f")}',
    "",
    f'거래량변동율(%) MIN: {format(new_df.amt_change.min(), ".2f")}',
    f'거래량변동율(%) MAX: {format(new_df.amt_change.max(), ".2f")}',
    f'거래량변동율(%) MEAN: {format(new_df.amt_change.mean(), ".2f")}',
    sep = '\n'
)

가격변동율(%) MIN: -5.68
가격변동율(%) MAX: 3.12
가격변동율(%) MEAN: -0.04

거래량변동율(%) MIN: -74.52
거래량변동율(%) MAX: 481.35
거래량변동율(%) MEAN: 11.69


In [28]:
## 양(+)의 가격변화율이 가장 높은 데이터 5개
new_df.sort_values(by=("price_change"), axis = 0, ascending=False).iloc[0:5]
    # 참고: 전부 다 3월임

,date,price,price_diff,price_change,amt,amt_diff,amt_change
190,2020.03.25,28912,876,3.12,1652775,148189,9.85
188,2020.03.27,30415,875,2.96,1785610,41062,2.35
205,2020.03.04,33552,784,2.39,1473186,745518,102.45
189,2020.03.26,29540,628,2.17,1744548,91773,5.55
196,2020.03.17,30890,641,2.12,1514025,-135462,-8.21


In [29]:
## 음(-)의 가격변화율이 가장 높은 데이터 5개
new_df.sort_values(by=("price_change"), axis = 0, ascending=True).iloc[0:5]
    # 참고: 12/29 제외하고, 전부 다 3월임

,date,price,price_diff,price_change,amt,amt_diff,amt_change
192,2020.03.23,27795,-1674,-5.68,1874198,-105171,-5.31
198,2020.03.13,30140,-1801,-5.64,4259132,2199448,106.79
1,2020.12.29,33200,-1800,-5.14,1280573,-457593,-26.33
194,2020.03.19,29540,-1278,-4.15,3402131,2424481,247.99
199,2020.03.12,31941,-1012,-3.07,2059684,879247,74.48


In [30]:
## 거래량이 가장 높은 데이터 10개
new_df.sort_values(by=("amt"), axis = 0, ascending=False).iloc[0:10]
    # 참고: 3/13, 3/19 포함

,date,price,price_diff,price_change,amt,amt_diff,amt_change
33,2020.11.12,35695,190,0.54,5685100,794471,16.24
34,2020.11.11,35505,42,0.12,4890629,4049369,481.35
9,2020.12.16,34910,530,1.54,4878990,3817867,359.79
32,2020.11.13,35170,-525,-1.47,4353288,-1331812,-23.43
198,2020.03.13,30140,-1801,-5.64,4259132,2199448,106.79
194,2020.03.19,29540,-1278,-4.15,3402131,2424481,247.99
239,2020.01.14,35252,-239,-0.67,3334474,993408,42.43
17,2020.12.04,34905,-140,-0.40,2910301,1643316,129.70
132,2020.06.19,32961,-31,-0.09,2878485,1994888,225.77
212,2020.02.24,33460,-722,-2.11,2703512,1360734,101.34
